In [2]:
!pip install streamlit
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 6.6 MB/s eta 0:00:00


In [3]:
!wget https://github.com/OmarAshraf772/OmarAbuzied/raw/refs/heads/main/Projects/depi/deployment.zip

--2024-11-08 18:37:32--  https://github.com/OmarAshraf772/OmarAbuzied/raw/refs/heads/main/Projects/depi/deployment.zip
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/OmarAshraf772/OmarAbuzied/refs/heads/main/Projects/depi/deployment.zip [following]
--2024-11-08 18:37:32--  https://raw.githubusercontent.com/OmarAshraf772/OmarAbuzied/refs/heads/main/Projects/depi/deployment.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11963087 (11M) [application/zip]
Saving to: ‘deployment.zip’

deployment.zip      100%[===================>]  11.41M  --.-KB/s    in 0.07s   

2024-11-08 18:37:33 (164 MB/s) 

In [5]:
import zipfile
zip_path = '/content/deployment.zip'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content')

In [6]:
%%writefile app.py
import zipfile
import pandas as pd
import joblib
import streamlit as st
#from sklearn.preprocessing import StandardScaler

# Load the model, encoder, scaler, and imputer
model = joblib.load('/content/xgb_model (1).pkl')
le = joblib.load('/content/encoders.pkl')
scaler = joblib.load('/content/standard_scaler (2).pkl')

# Ensure these variables are defined
data = pd.read_csv('/content/dataExtracted.csv')
VALID_STATE_ABBREVIATIONS = data['state'].unique().tolist()
VALID_JOBS = data['job'].unique().tolist()
VALID_CATEGORIES = data['category'].unique().tolist()

def categorize_numeric_features(input_df):
    input_df['Age_Category'] = pd.cut(input_df['age'],
                                       bins=[0, 18, 35, 55, float('inf')],
                                       labels=['Child', 'Young Adult', 'Adult', 'Senior'],
                                       right=False)
    input_df['population_cat'] = pd.cut(input_df['population'],
                                         bins=[0, 2500, 20000, 100000, float('inf')],
                                         labels=['Very Small', 'Small', 'Large', 'Very Large'],
                                         right=False)
    return input_df

def preprocess_user_input(user_input):
    input_df = pd.DataFrame([user_input])
    original_input_df = input_df.copy()
    input_df = categorize_numeric_features(input_df)
    input_df.drop(columns=['age', 'population'], inplace=True, errors='ignore')

    for column in ['Hour_Category', 'Displacement_Category', 'state', 'gender', 'job', 'category', 'Age_Category', 'population_cat']:
        if column in le:
            input_df[column] = le[column].transform(input_df[column])

    if 'amt' in input_df.columns:
        input_df[['amt']] = scaler.transform(input_df[['amt']])

    expected_feature_order = ['category', 'amt', 'gender', 'state', 'job',
                              'Hour_Category', 'Age_Category', 'Displacement_Category',
                              'population_cat']
    input_df = input_df.reindex(columns=expected_feature_order)
    return original_input_df, input_df

def make_prediction(user_input):
    original_input_df, processed_input = preprocess_user_input(user_input)
    prediction = model.predict(processed_input)
    prediction_result = "Fraud" if prediction[0] == 1 else "Not Fraud"
    return prediction_result, original_input_df, processed_input

# Define Streamlit application
st.title("Fraud Prediction App")

amt = st.number_input("Transaction Amount")
Hour_Category = st.selectbox("Hour Category", options=['Early Morning', 'Morning', 'Afternoon', 'Evening'])
age = st.number_input("Age")
population = st.number_input("City Population")
Displacement_Category = st.selectbox("Displacement Category", options=['Very Close', 'Close', 'Moderate', 'Far'])
state = st.selectbox("State Abbreviation", options=VALID_STATE_ABBREVIATIONS)
gender = st.selectbox("Gender", options=['M', 'F'])
job = st.selectbox("Job", options=VALID_JOBS)
other_job = st.text_input("If 'Other', please specify:")
category = st.selectbox("Transaction Category", options=VALID_CATEGORIES)

if job == "Other":
    job = other_job

if st.button("Predict"):
    user_input = {
        "category": category,
        "amt": amt,
        "gender": gender,
        "state": state,
        "job": job,
        "Hour_Category": Hour_Category,
        "age": age,
        "Displacement_Category": Displacement_Category,
        "population": population,
    }

    prediction_result, original_input_df, processed_input = make_prediction(user_input)
    st.warning(f"Prediction: {prediction_result}")

    # Save the original user input and processed input to CSV
    user_input_csv_path = "user_input_data.csv"
    processed_input_csv_path = "processed_input_data.csv"

    original_input_df.to_csv(user_input_csv_path, index=False)
    processed_input.to_csv(processed_input_csv_path, index=False)

    # Provide download buttons for the CSV files
    #with open(user_input_csv_path, "rb") as f:
     #   st.download_button(label="Download User Input CSV", data=f, file_name="user_input_data.csv")

    #with open(processed_input_csv_path, "rb") as f:
     #   st.download_button(label="Download Processed Input CSV", data=f, file_name="processed_input_data.csv")

Writing app.py


In [7]:
! wget -q -O - ipv4.icanhazip.com

35.236.241.15


In [ ]:
! streamlit run app.py & npx localtunnel --port 8501



Need to install the following packages:
  localtunnel@2.0.2
Ok to proceed? (y) 
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.236.241.15:8501

y
your url is: https://plain-guests-end.loca.lt
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [18:39:16] WARNING: /workspace/src/collective/../data/../common/error_msg.h:80: If you are loading a serialized model (like pickle in Python, RDS in R) or
configuration generated by an older version of XGBoost, please export the model by calling
`Booster.save_model` from that version first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/stable/tutorials/saving_model.html

for more details about differences between saving model and serializing.

  warnings.warn(smsg, UserWarning)
